In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import warnings

from futsim_funcs import TPR,TFR,TPSR,TGKR,TDR

In [2]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)

<br>

In [3]:
players_df = pd.read_csv("../fifa24_db/pdb_23.csv")
players_df = players_df

FileNotFoundError: [Errno 2] No such file or directory: '../fifa24_db/pdb_23.csv'

# futsim_funcs.py

In [ ]:
def TPR(df, top_N_players=18, lineup=False):
    '''
    Team Power Rating
    '''
    club_groups = df.groupby('club_name')
    dfs = []
    for club_name, club_data in club_groups:
        top_players = club_data.sort_values(by='overall', ascending=False).head(top_N_players)
        mean_top_players = top_players.mean(numeric_only=True)
        filler = int(mean_top_players['overall'])

        if lineup == True:
            top_players = top_players[~top_players['club_position'].str.contains('SUB|RES')]
        
        goalkeepers = top_players[top_players['player_positions'].str.contains('GK')].mean(numeric_only=True).fillna(filler)
        defenders = top_players[top_players['player_positions'].str.contains('CB|LB|RB|LWB|RWB')].mean(numeric_only=True).fillna(filler)
        midfielders = top_players[top_players['player_positions'].str.contains('CM|CDM|CAM|LM|RM')].mean(numeric_only=True).fillna(filler)
        attackers = top_players[top_players['player_positions'].str.contains('ST|CF|LW|RW|LF|RF')].mean(numeric_only=True).fillna(filler)
        midcenters = top_players[top_players['player_positions'].str.contains('CM|CDM')].mean(numeric_only=True).fillna(filler)
        wingers = top_players[top_players['player_positions'].str.contains('LB|RB|LWB|RWB|LM|RM|LW|RW|LF|RF')].mean(numeric_only=True).fillna(filler)
        
        club_df = pd.DataFrame({
            "league_id": top_players.iloc[0]['league_id'],
            "club_team_id": top_players.iloc[0]['club_team_id'],
            "league_name": top_players.iloc[0]['league_name'],
            "club_name": club_name,
            "def_power": int(goalkeepers['overall']*0.2 + defenders['overall']*0.8), # DEF POWER FUNCTION
            "mid_power": int(midfielders['overall']), # MID POWER FUNCTION
            "att_power": int(attackers['overall']), # ATT POWER FUNCTION
            "power": int(mean_top_players['overall']),
        }, index=[0])
        
        attributes = [
            'attacking_crossing','attacking_finishing', 'attacking_heading_accuracy',
           'attacking_short_passing', 'attacking_volleys', 'skill_dribbling',
           'skill_curve', 'skill_fk_accuracy', 'skill_long_passing',
           'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed',
           'movement_agility', 'movement_reactions', 'movement_balance',
           'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength',
           'power_long_shots', 'mentality_aggression', 'mentality_interceptions',
           'mentality_positioning', 'mentality_vision', 'mentality_penalties',
           'mentality_composure', 'defending_marking_awareness',
           'defending_standing_tackle', 'defending_sliding_tackle',
           'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
           'goalkeeping_positioning', 'goalkeeping_reflexes'
        ]

        mean_data = {}
        for position, players in zip(['goalkeepers', 'defenders', 'midfielders', 'attackers', 'midcenters', 'wingers'], 
                                     [goalkeepers, defenders, midfielders, attackers, midcenters, wingers]):
            for attribute in attributes:
                mean_data[f"{position}_{attribute}"] = int(players[attribute]) if not pd.isnull(players[attribute]) else 70

        for attribute in attributes:
            mean_data[f"team_{attribute}"] = int(players[attribute]) if not pd.isnull(players[attribute]) else 70

        mean_df = pd.DataFrame(mean_data, index=[0]) 
        club_df = pd.concat([club_df, mean_df], axis=1)
        
        dfs.append(club_df)
    merged = pd.concat(dfs, ignore_index=True)

    return {
        "name": "power",
        "data": merged.sort_values("power", ascending=False)
    }

In [ ]:
def TFNR(df, tpr_data):
    '''
    Team Finishing Rating
    '''
    club_groups = df.groupby('club_name')
    dfs = []
    for club_name, club_data in club_groups:
        tpr_data_club  = tpr_data.query(f"club_name == '{club_name}'")

        power = tpr_data_club["power"].iloc[0]
        aap = tpr_data_club["attackers_mentality_positioning"].iloc[0]
        aaf = tpr_data_club["attackers_attacking_finishing"].iloc[0]
        mpsp = tpr_data_club["midfielders_power_shot_power"].iloc[0]
        mls = tpr_data_club["midfielders_power_long_shots"].iloc[0]
        mha = tpr_data_club["defenders_attacking_heading_accuracy"].iloc[0]
        msc = tpr_data_club["midfielders_skill_curve"].iloc[0]
        asc = tpr_data_club["attackers_skill_curve"].iloc[0]
        
        skill_rating = int(
                            aap * 0.15 +
                            aaf * 0.30 +
                            mpsp * 0.20 +
                            mls * 0.20 +
                            msc * 0.05 +
                            asc * 0.05 +
                            mha * 0.05
                        )

        club_df = pd.DataFrame({
            "league_id": tpr_data_club.iloc[0]['league_id'],
            "club_team_id": tpr_data_club.iloc[0]['club_team_id'],  
            "league_name": tpr_data_club.iloc[0]['league_name'],
            "club_name": club_name,
            "power": power,
            "finishing": skill_rating
        }, index=[0])

        dfs.append(club_df)
    merged = pd.concat(dfs, ignore_index=True)

    return {
        "name": "finishing",
        "data": merged.sort_values("finishing", ascending=False)
    }

In [ ]:
def TPSR(df, tpr_data):
    '''
    Team Passing Rating
    '''
    club_groups = df.groupby('club_name')
    dfs = []
    for club_name, club_data in club_groups:
        tpr_data_club  = tpr_data.query(f"club_name == '{club_name}'")

        power = tpr_data_club["power"].iloc[0]
        short_passing = tpr_data_club["team_attacking_short_passing"].iloc[0]
        long_passing = tpr_data_club["team_skill_long_passing"].iloc[0]
        crossing = tpr_data_club["team_attacking_crossing"].iloc[0]
        vision = tpr_data_club["team_mentality_vision"].iloc[0]
        ball_control = tpr_data_club["team_skill_ball_control"].iloc[0]

        skill_rating = int( 
                            short_passing * 0.35 +
                            long_passing * 0.15 +
                            crossing * 0.10 +
                            vision * 0.20 +
                            ball_control * 0.20
                        )

        club_df = pd.DataFrame({
            "league_id": tpr_data_club.iloc[0]['league_id'],
            "club_team_id": tpr_data_club.iloc[0]['club_team_id'],  
            "league_name": tpr_data_club.iloc[0]['league_name'],
            "club_name": club_name,
            "power": power,
            "passing": skill_rating
        }, index=[0])

        dfs.append(club_df)
    merged = pd.concat(dfs, ignore_index=True)

    return {
        "name": "passing",
        "data": merged.sort_values("passing", ascending=False)
    }    

In [ ]:
def TDRR(df, tpr_data):
    '''
    Team Dribbling Rating
    '''
    club_groups = df.groupby('club_name')
    dfs = []
    for club_name, club_data in club_groups:
        tpr_data_club  = tpr_data.query(f"club_name == '{club_name}'")

        power = tpr_data_club["power"].iloc[0]
        speed = tpr_data_club["team_movement_sprint_speed"].iloc[0]
        balance = tpr_data_club["team_movement_balance"].iloc[0]
        att_acceleration = tpr_data_club["attackers_movement_acceleration"].iloc[0]
        att_dribbling = tpr_data_club["attackers_skill_dribbling"].iloc[0]
        mid_acceleration = tpr_data_club["midfielders_movement_acceleration"].iloc[0]
        mid_dribbling = tpr_data_club["midfielders_skill_dribbling"].iloc[0]

        skill_rating = int( 
                            speed * 0.15 +
                            balance * 0.15 +
                            att_acceleration * 0.15 +
                            att_dribbling * 0.20 +
                            mid_acceleration * 0.15 +
                            mid_dribbling * 0.20
                        )

        club_df = pd.DataFrame({
            "league_id": tpr_data_club.iloc[0]['league_id'],
            "club_team_id": tpr_data_club.iloc[0]['club_team_id'],  
            "league_name": tpr_data_club.iloc[0]['league_name'],
            "club_name": club_name,
            "power": power,
            "dribbling": skill_rating
        }, index=[0])

        dfs.append(club_df)
    merged = pd.concat(dfs, ignore_index=True)

    return {
        "name": "dribbling",
        "data": merged.sort_values("dribbling", ascending=False)
    }    

In [ ]:
def TDER(df, tpr_data):
    '''
    Team Defending Rating
    '''
    club_groups = df.groupby('club_name')
    dfs = []
    for club_name, club_data in club_groups:
        tpr_data_club  = tpr_data.query(f"club_name == '{club_name}'")
        
        power = tpr_data_club["power"].iloc[0]
        def_speed = tpr_data_club["defenders_movement_sprint_speed"].iloc[0]
        mc_speed = tpr_data_club["midcenters_movement_sprint_speed"].iloc[0]
        def_int = tpr_data_club["defenders_mentality_interceptions"].iloc[0]
        mc_int = tpr_data_club["midcenters_mentality_interceptions"].iloc[0]
        def_standing_tackle = tpr_data_club["defenders_defending_standing_tackle"].iloc[0]
        mc_standing_tackle = tpr_data_club["midcenters_defending_standing_tackle"].iloc[0]     
        def_sliding_tackle = tpr_data_club["defenders_defending_sliding_tackle"].iloc[0]
        mc_sliding_tackle = tpr_data_club["midcenters_defending_sliding_tackle"].iloc[0]    
        marking = tpr_data_club["team_defending_sliding_tackle"].iloc[0]    
        agression = tpr_data_club["team_mentality_aggression"].iloc[0]    

        skill_rating = int( 
                            def_speed * 0.07 +
                            mc_speed * 0.07 +
                            def_int * 0.10 +
                            mc_int * 0.06 +
                            def_standing_tackle * 0.20 +
                            mc_standing_tackle * 0.05 +
                            def_sliding_tackle * 0.15 +
                            mc_sliding_tackle * 0.05 +
                            marking * 0.17 +
                            agression * 0.08
                          )

        club_df = pd.DataFrame({
            "league_id": tpr_data_club.iloc[0]['league_id'],
            "club_team_id": tpr_data_club.iloc[0]['club_team_id'],  
            "league_name": tpr_data_club.iloc[0]['league_name'],
            "club_name": club_name,
            "power": power,
            "defending": skill_rating
        }, index=[0])

        dfs.append(club_df)
    merged = pd.concat(dfs, ignore_index=True)

    return {
        "name": "defending",
        "data": merged.sort_values("defending", ascending=False)
    }    

In [ ]:
def TGKR(df, tpr_data):
    '''
    Team Goalkeeping Rating
    '''
    club_groups = df.groupby('club_name')
    dfs = []
    for club_name, club_data in club_groups:
        tpr_data_club  = tpr_data.query(f"club_name == '{club_name}'")
        
        power = tpr_data_club["power"].iloc[0]
        diving = tpr_data_club["goalkeepers_goalkeeping_diving"].iloc[0]    
        handling = tpr_data_club["goalkeepers_goalkeeping_handling"].iloc[0]    
        kicking = tpr_data_club["goalkeepers_goalkeeping_kicking"].iloc[0]    
        positioning = tpr_data_club["goalkeepers_goalkeeping_positioning"].iloc[0]    
        reflexes = tpr_data_club["goalkeepers_goalkeeping_reflexes"].iloc[0]    

        skill_rating = int( 
                            diving * 0.25 +
                            handling * 0.20 +
                            kicking * 0.10 +
                            positioning * 0.25 +
                            reflexes * 0.20
                          )

        club_df = pd.DataFrame({
            "league_id": tpr_data_club.iloc[0]['league_id'],
            "club_team_id": tpr_data_club.iloc[0]['club_team_id'],  
            "league_name": tpr_data_club.iloc[0]['league_name'],
            "club_name": club_name,
            "power": power,
            "goalkeeping": skill_rating
        }, index=[0])

        dfs.append(club_df)
    merged = pd.concat(dfs, ignore_index=True)

    return {
        "name": "goalkeeping",
        "data": merged.sort_values("goalkeeping", ascending=False)
    }

In [ ]:
def TSTR(df, tpr_data):
    '''
    Team Strength Rating
    '''
    club_groups = df.groupby('club_name')
    dfs = []
    for club_name, club_data in club_groups:
        tpr_data_club  = tpr_data.query(f"club_name == '{club_name}'")

        power = tpr_data_club["power"].iloc[0]
        strength = tpr_data_club["team_power_strength"].iloc[0]    
        speed = tpr_data_club["team_movement_sprint_speed"].iloc[0]    
        agility = tpr_data_club["team_movement_agility"].iloc[0]    
        skill_rating = int( 
                            strength * 25 +
                            speed * 0.2 + 
                            agility * 0.1
                          )

        club_df = pd.DataFrame({
            "league_id": tpr_data_club.iloc[0]['league_id'],
            "club_team_id": tpr_data_club.iloc[0]['club_team_id'],  
            "league_name": tpr_data_club.iloc[0]['league_name'],
            "club_name": club_name,
            "power": power,
            "strength": skill_rating
        }, index=[0])

        dfs.append(club_df)
    merged = pd.concat(dfs, ignore_index=True)

    return {
        "name": "strength",
        "data": merged.sort_values("strength", ascending=False)
    }

In [ ]:
def TSMR(df, tpr_data):
    '''
    Team Stamina Rating
    '''
    club_groups = df.groupby('club_name')
    dfs = []
    for club_name, club_data in club_groups:
        tpr_data_club  = tpr_data.query(f"club_name == '{club_name}'")
        
        power = tpr_data_club["power"].iloc[0]
        stamina = tpr_data_club["team_power_stamina"].iloc[0]    

        skill_rating = int( 
                            stamina * 1
                          )

        club_df = pd.DataFrame({
            "league_id": tpr_data_club.iloc[0]['league_id'],
            "club_team_id": tpr_data_club.iloc[0]['club_team_id'],  
            "league_name": tpr_data_club.iloc[0]['league_name'],
            "club_name": club_name,
            "power": power,
            "stamina": skill_rating
        }, index=[0])

        dfs.append(club_df)
    merged = pd.concat(dfs, ignore_index=True)

    return {
        "name": "stamina",
        "data": merged.sort_values("stamina", ascending=False)
    }

In [ ]:
def THER(df, tpr_data):
    '''
    Team Heading Rating
    '''
    club_groups = df.groupby('club_name')
    dfs = []
    for club_name, club_data in club_groups:
        tpr_data_club  = tpr_data.query(f"club_name == '{club_name}'")
        
        power = tpr_data_club["power"].iloc[0]
        heading = tpr_data_club["team_attacking_heading_accuracy"].iloc[0]    
        jumping = tpr_data_club["team_power_jumping"].iloc[0]    

        skill_rating = int( 
                            heading * 0.7 +
                            jumping * 0.3
                          )

        club_df = pd.DataFrame({
            "league_id": tpr_data_club.iloc[0]['league_id'],
            "club_team_id": tpr_data_club.iloc[0]['club_team_id'],  
            "league_name": tpr_data_club.iloc[0]['league_name'],
            "club_name": club_name,
            "power": power,
            "heading": skill_rating
        }, index=[0])

        dfs.append(club_df)
    merged = pd.concat(dfs, ignore_index=True)

    return {
        "name": "heading",
        "data": merged.sort_values("heading", ascending=False)
    }

# New Rating Functions

In [ ]:
tpr_data = TPR(players_df, top_N_players=33333, lineup=False)["data"]
tpr_data.sort_values("power", ascending=False)[["club_name","power","def_power", "mid_power", "att_power"]].head(5)

f = TSTR
test_data = f(players_df, tpr_data)["data"]
test_data.sort_values("strength", ascending=False).head()

In [ ]:
np.random.choice(["Goal", "No"], p=[0.25, 0.75])

In [ ]:
players_df.columns